In [7]:
# %jsroot on
# different distribution of mc data,inclusive for plots in note for Dijet
# add all Central/Forward LeadingJet/SubJet
from ROOT import *
import numpy as np
import os

pt_bin_array = np.asarray([500, 600, 800, 1000, 1200, 1500, 2000])


jet_type_array = ["LeadingJet","SubJet"]
var_array = ["eta","ntrk","c1","width","bdt"]
var_title_array = ["#eta","N_{trk}","C_{1}^{#beta=0.2}","W_{trk}", "BDT"]
region_array = ["_Central", "_Forward"]

def myText(x,y,text, color=1):
    l= TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    
def getRelErrors(h):
    h_err = h.Clone()
    h_err.SetDirectory(0)
    h_err.Reset()

    for bin in range(h.GetNbinsX()+1):
        content = h.GetBinContent(bin)
        error = h.GetBinError(bin)
        if(content != 0):
            h_err.SetBinContent(bin, 0)
            h_err.SetBinError(bin, error / content)
    
    return h_err
        
#   h.h_bkgShapeSys->SetMarkerSize(0);
#   h.h_bkgShapeSys->SetLineColor(4);
#   h.h_bkgShapeSys->SetLineWidth(2);
#   h.h_bkgShapeSys->SetFillColor(0);
#   h.h_bkgShapeSys->SetFillStyle(0);
#   drawErrorBand(h.h_bkgShapeSys);

        
def drawErrorBand(h):
    h_Up = h.Clone()
    h_Do = h.Clone()
    h_Up.Reset()
    h_Do.Reset()
    for bin in range(1, h.GetNbinsX()):
        content = h.GetBinContent(bin);
        error = h.GetBinError(bin);
        h_Up.SetBinContent(bin, content+error);
        h_Do.SetBinContent(bin, content-error);
#     myText(0.15,0.15,"DrawErrorBand")
    h_Up.Draw("same");
    h_Do.Draw("same");

mc_type = "pythia"
dijet_mc_file = TFile("../newroot/dijet_pythia_nominal.root")
dijet_data_file = TFile("../newroot/dijet_data_prescale.root")

gStyle.SetOptStat(0)

var = var_array[0]
jet_type = jet_type_array[0]
region = region_array[0]

pt_bin=pt_bin_array[0]
var_title=var_title_array[0]

path = mc_type+"_vs_data"
isExists=os.path.exists(mc_type+"_vs_data")
if not isExists:
        os.makedirs(mc_type+"_vs_data") 
        
for region in region_array:
    for j,var in enumerate(var_array):
        MC_per_pt_bin = []
        MC_error_per_pt_bin = []
        Data_per_pt_bin = []
        for pt_i, pt_bin in enumerate(pt_bin_array[:-1]):
            Quark_leading = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Quark_" + var)
            Gluon_leading = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Gluon_" + var)
            Other_leading = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Other_" + var)
            Quark_leading_error = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Quark_" + var + "_err")
            Gluon_leading_error = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Gluon_" + var + "_err")
            Other_leading_error = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Other_" + var + "_err")

            Quark_subleading = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Quark_" + var)
            Gluon_subleading = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Gluon_" + var)
            Other_subleading = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Other_" + var)
            Quark_subleading_error = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Quark_" + var + "_err")
            Gluon_subleading_error = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Gluon_" + var + "_err")
            Other_subleading_error = dijet_mc_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Other_" + var + "_err")


            for i in range(1, Quark_leading.GetNbinsX()+1):
                Quark_leading.SetBinError(i,np.sqrt(Quark_leading_error.GetBinContent(i)))
                Gluon_leading.SetBinError(i,np.sqrt(Gluon_leading_error.GetBinContent(i)))
                Other_leading.SetBinError(i,np.sqrt(Other_leading_error.GetBinContent(i)))

                Quark_subleading.SetBinError(i,np.sqrt(Quark_subleading_error.GetBinContent(i)))
                Gluon_subleading.SetBinError(i,np.sqrt(Gluon_subleading_error.GetBinContent(i)))
                Other_subleading.SetBinError(i,np.sqrt(Other_subleading_error.GetBinContent(i)))

            Data_leading = dijet_data_file.Get(str(pt_bin)+ "_" + jet_type_array[0] + region + "_Data_" + var)
            Data_subleading = dijet_data_file.Get(str(pt_bin)+ "_" + jet_type_array[1] + region + "_Data_" + var)


            Quark_leading.Add(Quark_subleading)
            Gluon_leading.Add(Gluon_subleading)
            Other_leading.Add(Other_subleading)

            MC = Quark_leading.Clone()
            MC.Add(Gluon_leading)
            MC.Add(Other_leading)

            Data_leading.Add(Data_subleading)

            MC_error = getRelErrors(MC)

            MC_per_pt_bin.append(MC)
            MC_error_per_pt_bin.append(MC_error)
            Data_per_pt_bin.append(Data_leading)
            

        MC_total = MC_per_pt_bin[0].Clone()
        Data_total = Data_per_pt_bin[0].Clone()

        for i in range(1, len(MC_per_pt_bin)):
            MC_total.Add(MC_per_pt_bin[i])
            Data_total.Add(Data_per_pt_bin[i])
        

        for i in range(len(MC_per_pt_bin)):
            MC_per_pt_bin[i].Scale(1./MC_per_pt_bin[i].Integral()) 
            Data_per_pt_bin[i].Scale(1./Data_per_pt_bin[i].Integral()) 

        MC_total.Scale(1./MC_total.Integral()) 
        Data_total.Scale(1./Data_total.Integral()) 

        
        for pt_i, pt_bin in enumerate(pt_bin_array[:-1]): 
            MC_500_600 = MC_per_pt_bin[pt_i].Clone()
    #         MC_800_1000 = MC_per_pt_bin[2].Clone()

            MC_error_500_600 = MC_error_per_pt_bin[pt_i].Clone()
            Data_500_600 = Data_per_pt_bin[pt_i].Clone()
    #         Data_800_1000 = Data_per_pt_bin[2].Clone()

            c = TCanvas("","",800,600)
            c.Divide(2,1)

            c.cd()
            pad1 = TPad("pad1", "pad1", 0.0,0.3,1.0,1.0)
            pad1.SetBottomMargin(0)
            pad1.SetGridx()
            pad1.Draw()
            pad1.cd()

    #         MC_total.SetLineColor(1)
    #         Data_total.SetLineColor(1)

            MC_500_600.SetLineColor(2)
    #         Data_500_600.SetLineColor(4)



            YMaximum = MC_500_600.GetMaximum() if MC_500_600.GetMaximum() > Data_500_600.GetMaximum() else Data_500_600.GetMaximum() 

            MC_500_600.GetYaxis().SetRangeUser(0., YMaximum * 1.5)
            MC_500_600.GetYaxis().SetTitle("Normalized to unity")

    #         MC_total.Draw("HIST")
            MC_500_600.Draw("HISTE")

            Data_500_600.SetMarkerStyle(8)
            Data_500_600.SetMarkerColor(4)
    #         Data_500_600.SetMarkerStyle(8)

    #         Data_total.Draw("E1P same")
            Data_500_600.Draw("E0 same")


            leg1 = TLegend(.7,.65,.9,.85)
    #         leg1.AddEntry(MC_total,"MC MC","L")
    #         leg1.AddEntry(Data_total,"Data","LEP")
            leg1.AddEntry(MC_500_600,str(pt_bin_array[pt_i])+"<p_{T}<"+str(pt_bin_array[pt_i+1])+" GeV, MC","l")
            leg1.AddEntry(Data_500_600,str(pt_bin_array[pt_i])+"<p_{T}<"+str(pt_bin_array[pt_i+1])+ "GeV, Data","ep")

            leg1.SetBorderSize(0)
            leg1.Draw("same")

            myText(0.15,0.85,"#it{#bf{#scale[2]{#bf{ATLAS} Simulation Internal}}}")
            myText(0.15,0.80,"#bf{#scale[1.8]{#sqrt{s}=13 TeV}}")
            myText(0.15,0.75,"#bf{#scale[1.8]{Anti-K_{t} EM+JES R=0.4}}")
            myText(0.15,0.70,"#bf{#scale[1.8]{" + region[1:] + " Jets}}")


            c.cd()
            pad2= TPad("pad2", "pad2", 0.0,0.05,1.0,0.3)
            pad2.SetTopMargin(0.)
            pad2.SetBottomMargin(0.3)

            pad2.Draw()
            pad2.cd()
    #         total_ratio = Data_total.Clone()
            sub_500_600_ratio = Data_500_600.Clone()

    #         total_ratio.Divide(MC_total)
            sub_500_600_ratio.Divide(MC_500_600)
    #         sub_800_1000_ratio.Divide(MC_800_1000)


            sub_500_600_ratio.GetYaxis().SetRangeUser(0.7,1.3)
            sub_500_600_ratio.GetXaxis().SetTitle(var_title_array[j])
            # total_ratio.GetXaxis().SetTitleOffset(0.5)
            sub_500_600_ratio.GetXaxis().SetTitleOffset(0.8)
            sub_500_600_ratio.GetXaxis().SetTitleSize(0.15)
            sub_500_600_ratio.GetXaxis().SetLabelSize(0.1)

            # total_ratio.GetXaxis().SetLabelOffset(0.03)

            sub_500_600_ratio.GetYaxis().SetTitleOffset(0.3)
            # total_ratio.GetYaxis().SetLabelOffset(0.01)

            sub_500_600_ratio.GetYaxis().SetLabelSize(0.1)
            sub_500_600_ratio.GetYaxis().SetTitle('Data/MC')
            sub_500_600_ratio.GetYaxis().SetTitleSize(0.1)

            sub_500_600_ratio.Draw("EP")
            MC_error_500_600.SetMarkerSize(0)
            MC_error_500_600.SetLineColor(4)
            MC_error_500_600.SetLineWidth(2)
            MC_error_500_600.SetFillColor(0)
            MC_error_500_600.SetFillStyle(0)
#             drawErrorBand(MC_error_total)
    #         sub_500_600_ratio.Draw("P same")
    #         sub_800_1000_ratio.Draw("P same")

            line = TLine(sub_500_600_ratio.GetXaxis().GetXmin(),1,sub_500_600_ratio.GetXaxis().GetXmax(),1)
            line.SetLineStyle(7)
            line.SetLineWidth(7)
            line.SetLineColor(12)
            line.Draw("same")
            
    #         c.Draw()
            c.Print(mc_type+"_vs_data/"+mc_type +"-"+ var + region+"_"+str(pt_bin_array[pt_i])+"_"+str(pt_bin_array[pt_i+1])+".pdf")

Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-eta_Central_500_600.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-eta_Central_600_800.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-eta_Central_800_1000.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-eta_Central_1000_1200.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-eta_Central_1200_1500.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-eta_Central_1500_2000.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-ntrk_Central_500_600.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-ntrk_Central_600_800.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-ntrk_Central_800_1000.pdf has been created
Info in <TCanvas::Print>: pdf file pythia_vs_data/pythia-ntrk_Central_1000_1200.pdf has been created
Info i